$\mathrm{\huge{Neolithic\,Circular\,Enclosures}}$

$\mathrm{\large{Notebook\,I:\, Calculation\,of\,star\,paths\,on\,the\,horizon}}$


$\mathrm{Gerd\,Graßhoff^{1,2,3}, Gordon\,Fischer^{2}}$
  
1: Humboldt Universität zu Berlin, 2: Excellence Cluster Topoi  
3: Max Planck Institute for history  of science  
Corresponding mail: gerd.grasshoff@hu-berlin.de, gordon.fischer@topoi.org  
Date: 18.05.2017

**goal of research:** Calculation of star positions (altitude, azimuth) on the horizon
        
Keywords: Neolithic Circular Enclosures, star position, altitude, azimuth

In [1]:
#IMPORT LIBRARIES
import DatabaseKreisgrabenanlage as bsc
import csv
import json
import astronomia
import ephem
from ephem import *
import numpy as np
from ephem import stars 
import pandas as pd
db=stars.db
import numpy as np
import math
import sys
from IPython.display import display
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import Counter
from ephem import Equatorial, Ecliptic
from IPython.display import Image, clear_output
from ipywidgets import FloatSlider,IntSlider,Dropdown,Label, Button, Layout, Textarea, HBox, VBox, ToggleButtons, RadioButtons, jslink,interact, interact_manual,Output, Text, FloatText, Select

ModuleNotFoundError: No module named 'DatabaseKreisgrabenanlage'

In [ ]:
#FUNCTIONS
stars = {}
global stars
for line in db.strip().split('\n'):
        star = ephem.readdb(line)
        stars[star.name] = star
brightstars = sorted(list(stars.keys()))
dfmap=bsc.CatalogueMappingNames
def MatchProperName(val):
    d = df['Proper name'].str.contains(val)
    index = [i for i, x in enumerate(d) if x]
    return df.iloc[index]
def GetCSV(path):
    list_points = []
    with open(path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            list_points.append(row) 
    return list_points;
def DumpJSON(filename, path):
 
    with open(path, 'w') as outfile:
        json.dump(filename, outfile)
    return;

def GetJSON(path):

    with open(path, encoding="iso-8859-15" ) as data_file:    
        data = json.load(data_file)
    return data;

In [ ]:
#STAR CATALOGUE
#starsforwidget = list(dfmap["Proper name"])
pyephembsc=GetCSV("PyEphemStarCatalogue.csv")
stars=[]
for j in range(len(pyephembsc)):
    stars.append(pyephembsc[j][0])
starsforwidget=stars;

def CheckStars(starname):
    try:
        selectedstar=ephem.star(starname)
        return starname
    except:
         #print("no star")
        pass    
correctedStarlist=[]
for j in range(len(starsforwidget)):
    v1 = CheckStars(starsforwidget[j])
    if v1 != "no star":
        correctedStarlist.append(v1)
starsforwidget = [x for x in correctedStarlist if x is not None]
starsforwidget.insert(0,"no star")

In [ ]:
#STAR FUNCTION
def AzimuthAltitude(year, deviation, starname, Anlage,  xmin, xmax, ymin, ymax, zoom):
    gatech = ephem.Observer()
    anlage=GetJSON(Anlage)
    # Korrektur der Werte
    profil=[]
    profilAz=[]
    profilAlt=[]
    az = anlage["profile"]["azimuth"]
    zenitwinkel= anlage["profile"]["altitude"]

    for j in range(len(az)):

        if  float(az[j])>=90 and float(az[j])<360:
            profil.append(((-1*float(az[j]))+360+90,90-float(zenitwinkel[j]) ))

        if float(az[j])<90:
            profil.append(((-1*float(az[j]))+90,90-float(zenitwinkel[j]) ))


    profil = sorted(profil,key=lambda l:l[0], reverse=False)
    profilAz=[row[0] for row in profil]
    profilAlt=[row[1] for row in profil]

    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
    templist=[]
    templist1=[]
    templist2=[]
    azimuthStar=[]
    altitudeStar=[]
    azimuth1Star=[]
    altitude1Star=[]
    azimuth2Star=[]
    altitude2Star=[]

    tryv=True
    for j in range(96):

        try:
            gatech.date = str(year)+"/1/1  "+str(j/4.)+":00:00"
            selectedstar=ephem.star(starname)
            selectedstar.compute(gatech)
            templist.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
            templist = sorted(templist,key=lambda l:l[0], reverse=False)
            azimuthStar = [row[0] for row in templist]
            altitudeStar = [row[1] for row in templist]

            year1=year+deviation
            gatech.date = str(year1)+"/1/1  "+str(j/4.)+":00:00"
            selectedstar=ephem.star(starname)
            selectedstar.compute(gatech)
            templist1.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
            templist1 = sorted(templist1,key=lambda l:l[0], reverse=False)
            azimuth1Star = [row[0] for row in templist1]
            altitude1Star = [row[1] for row in templist1]

            year2=year-deviation
            gatech.date = str(year2)+"/1/1  "+str(j/4.)+":00:00"
            selectedstar=ephem.star(starname)
            selectedstar.compute(gatech)
            templist2.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
            templist2 = sorted(templist2,key=lambda l:l[0], reverse=False)
            azimuth2Star = [row[0] for row in templist2]
            altitude2Star = [row[1] for row in templist2]

        except:
            if j == 1:
                print("Star not available on ephem library!")
            tryv=False
            pass

    if zoom==False  and tryv !=False:
        ymax=max( altitudeStar + altitude1Star+ altitude2Star)+2

        xmin = 0
        xmax = 360
        ymin = 0

    if zoom==True:
        xmin = xmin
        xmax = xmax
        ymin = ymin
        ymax = ymax

    if min(profilAlt)<0:
        ymin=min(profilAlt)-0.2
    else:
        ymin=0
    fig, ax = plt.subplots(1, figsize=(20,9))
    plt.xlim(xmin, xmax)
    if ymax>0:
        plt.ylim(ymin,ymax)
    if ymax<=0:
        plt.ylim(ymin,2)
    #Horizontprofil  correctur: 90.-x for x in 
    plt.plot(profilAz, profilAlt, 'k', label="visible horizon")
    plt.plot([0,360],[0,0] , 'k--', label="math. horizon")

    #Plot Tore
    authorlist= list(anlage["gate"].keys())
    colorlist = ["blue", "green", "orange"]
    for k in range(len(authorlist)):

        for m in range(len(anlage["gate"][authorlist[k]])):
            if k== 0:
                if m == 0:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor=colorlist[k],alpha=0.5 )
                else:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.5)
            if k==1:
                if m == 0:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )
                else:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax],facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )


    #Plot Special Features
    authorlist= list(anlage["specialfeatures"].keys())
    colorlist = ["lightblue",  "lightgreen"]

    for k in range(len(authorlist)):
        for m in range(len(anlage["specialfeatures"][authorlist[k]])):
            if k==1:
                if m == 0:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor=colorlist[k],alpha=0.7 )
                else:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.7)
            if k==0:
                if m == 0:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor="none", hatch="\\\\", edgecolor="black", linewidth=0.0)
                else:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor="none",hatch="\\\\", edgecolor="black", linewidth=0.0)

    plt.plot(azimuthStar,altitudeStar, 'b', label=starname+' year: '+str(year)+', mag: '+str(selectedstar.mag))
    plt.plot(azimuth1Star,altitude1Star , 'k--', label=starname+' year: '+str(year1)+', mag: '+str(selectedstar.mag))
    plt.plot(azimuth2Star,altitude2Star , 'r--', label=starname+' year: '+str(year2)+', mag: '+str(selectedstar.mag))

    plt.legend(bbox_to_anchor=(0.80, 0.97), loc=2, borderaxespad=0.)
    plt.xlabel('Azimuth [°]')
    plt.ylabel('Altitude [°]')
    plt.title("Object: "+anlage["object"]+'\nConstellation for years: '+str(year)+'+/- '+str(deviation))
    return plt.show()

In [ ]:
# Test Function I
#AzimuthAltitude(-4850, 100,  'Sirius',"Ippesheim.json", 2, 360,0,3,zoom=False)

## Widget I: star dynamics

- compare the star path of a certain star over two epochs

In [ ]:
# WIDGET I
Objects=["QuedlinburgI","QuedlinburgII", "Hopferstadt", "Bielen","Ippesheim","Ippesheim_2","Kyhna1", "Kyhna2", "Kyhna3", "Kyhna4", "Goseck", "Warburg-Daseburg","Bochum-Harpen", "Gneiding", "Kothingeichendorf", "Watenstedt", "Dresden-NickernI", "Dresden-NickernII",  "Bochow", "Eythra2", "Dresden3", "Kuenzing-Unternberg", "Meisternthal", "Mirskofen", "Ramsdorf", "Schmierdorf1", "Schmierdorf2", "Stephansposching", "Viecht", "Koetschlitz"]
import warnings
warnings.filterwarnings('ignore')

out = Output()
par1=Dropdown(
    options=Objects,
    description='Object'
)
par2=Text(
    continuous_update=False,value="-4700",description='year',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par3=Text(
    continuous_update=False,value="20",description='+/-',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par5= Dropdown(
    continuous_update=False,options=starsforwidget,description='star',value="Sirius",layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par6xmin=IntSlider(
    value=0,min=0,max=360,step=1,description='az. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6xmax=IntSlider(
    value=360,min=0,max=360,step=1,description='az. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymin=IntSlider(
    value=0,min=-2,max=90,step=1,description='alt. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymax=IntSlider(
    value=3,min=0,max=90,step=1,description='alt. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par7=Button(
   description='RUN '
)
par8=Button(
   description='RUN with Zoom'
)
def SearchConstellation(res):
    firstpar =par1.value
    secondpar = par2.value
    thirdpar=par3.value
    fivthpar=par5.value
    objectfile=firstpar+".json"
    with out:
        clear_output()
        AzimuthAltitude(int(secondpar), int(thirdpar), str(fivthpar), objectfile, 0,360,0,90, zoom=False)
    return; 
    
def SearchConstellationZoom(res):
    firstpar =par1.value
    secondpar = par2.value
    thirdpar=par3.value
    fivthpar=par5.value
    sixthxmin=par6xmin.value
    sixthxmax=par6xmax.value
    sixthymin=par6ymin.value
    sixthymax=par6ymax.value
    objectfile=firstpar+".json"
    with out:
        clear_output()
        AzimuthAltitude(int(secondpar), int(thirdpar), str(fivthpar), objectfile, sixthxmin, sixthxmax, sixthymin, sixthymax, zoom=True)
    return; 
par7.on_click(SearchConstellation)
par8.on_click(SearchConstellationZoom)
display(VBox([Label('Select object:'), par1,Label('Select epochs'),HBox([par2, par3]), Label('Select star'), par5, par7, Label('Define zoom range'), HBox([par6xmin, par6xmax,par6ymin,par6ymax]),par8 ]))
display(out)

## Widget II: sun, moon and stars

In [ ]:
# FUNCTION FOR WIDGET II
def AzimuthAltitudeFull(date, starname1,starname2, starname3, Anlage, special, xmin, xmax, ymin, ymax, pressureval, zoom):
    gatech = ephem.Observer()
    gatech.pressure=pressureval
    anlage=GetJSON(Anlage)
    # Korrektur der Werte
    profil=[]
    profilAz=[]
    profilAlt=[]
    az = anlage["profile"]["azimuth"]
    zenitwinkel= anlage["profile"]["altitude"]

    for j in range(len(az)):
   
        if  float(az[j])>=90 and float(az[j])<360:
            profil.append(((-1*float(az[j]))+360+90,90-float(zenitwinkel[j]) ))
            
        if float(az[j])<90:
            profil.append(((-1*float(az[j]))+90,90-float(zenitwinkel[j]) ))
            

    profil = sorted(profil,key=lambda l:l[0], reverse=False)
    profilAz=[row[0] for row in profil]
    profilAlt=[row[1] for row in profil]
    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
    azimuthStar1=[]
    altitudeStar1=[]
    azimuthStar2=[]
    altitudeStar2=[]
    azimuthStar3=[]
    altitudeStar3=[]
    azimuthMoon=[]
    altitudeMoon=[]
    azimuthSun=[]
    altitudeSun=[]
    
    p = Equatorial('90', '0', epoch=str(date.split("/")[0]))
    eclipt= float(Ecliptic(p).lat)*180/3.14
   
    
    if special != "no selection":
        date= ephem.date((int(date.split("/")[0]),1,1))
        orbitPoints= ['vernal_equinox','summer_solstice','autumnal_equinox','winter_solstice']
        dates= {}
        dates['vernal_equinox_start']= ephem.next_vernal_equinox(date)
        
        if special=="vernal_equinox":
            date= ephem.next_vernal_equinox(date)
        if special=="summer_solstice":
            date= ephem.next_summer_solstice(dates['vernal_equinox_start'])
        if special=="autumnal_equinox":
            date= ephem.next_autumnal_equinox(dates['vernal_equinox_start'])
        if special=="winter_solstice":
            date = ephem.next_winter_solstice(dates['vernal_equinox_start'])
        if special =="all special events":
            dateFP = ephem.next_vernal_equinox(date)
            dateFP= str(dateFP).split(" ")[0]
            dateSSW = ephem.next_summer_solstice(dates['vernal_equinox_start'])
            dateSSW= str(dateSSW).split(" ")[0]
            dateHP = ephem.next_autumnal_equinox(dates['vernal_equinox_start'])
            dateHP= str(dateHP).split(" ")[0]
            dateWSW =ephem.next_winter_solstice(dates['vernal_equinox_start'])
            dateWSW= str(dateWSW).split(" ")[0]
           
       
        date=str(date).split(" ")[0]
    else: 
        date=date
    
    templist1=[]
    templist2=[]
    templist3=[]
    for j in range(96):
        
        gatech.date =str(date) +' '+str(j/4.)+":00:00"
        #Star
        if starname1 != "no star":
            try: 
                ephem.star(starname1)
                selectedstar=ephem.star(starname1)
                selectedstar.compute(gatech)
                mag1=selectedstar.mag
                templist1.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist1 = sorted(templist1,key=lambda l:l[0], reverse=False)
                azimuthStar1 = [row[0] for row in templist1]
                altitudeStar1 = [row[1] for row in templist1]
               

            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
     
        if starname2 != "no star":
            try: 
                ephem.star(starname2)
                selectedstar=ephem.star(starname2)
                selectedstar.compute(gatech)
                mag2=selectedstar.mag
                templist2.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist2 = sorted(templist2,key=lambda l:l[0], reverse=False)
                azimuthStar2 = [row[0] for row in templist2]
                altitudeStar2 = [row[1] for row in templist2]
            
            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
            
        if starname3 != "no star":
            try: 
                ephem.star(starname3)
                selectedstar=ephem.star(starname3)
                selectedstar.compute(gatech)
                mag3=selectedstar.mag
                templist3.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist3 = sorted(templist3,key=lambda l:l[0], reverse=False)
                azimuthStar3 = [row[0] for row in templist3]
                altitudeStar3 = [row[1] for row in templist3]
             

            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
        #Moon
        moon=ephem.Moon()
        moon.compute(gatech)
        azimuthMoon.append(float(moon.az*180/3.14))
        altitudeMoon.append(float(moon.alt*180/3.14))
        #Sun
        sun=ephem.Sun()
        sun.compute(gatech)
        azimuthSun.append(float(sun.az*180/3.14))
        altitudeSun.append(float(sun.alt*180/3.14))
    
    azimuthSunFP=[]
    altitudeSunFP=[]
    azimuthSunSSW=[]
    altitudeSunSSW=[]
    azimuthSunHP=[]
    altitudeSunHP=[]
    azimuthSunWSW=[]
    altitudeSunWSW=[]
    
    if special=="all special events":
        for j in range(96):
            gatech.date =str(dateFP) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunFP.append(float(sun.az*180/3.14))
            altitudeSunFP.append(float(sun.alt*180/3.14))
            gatech.date =str(dateSSW) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunSSW.append(float(sun.az*180/3.14))
            altitudeSunSSW.append(float(sun.alt*180/3.14))
            gatech.date =str(dateHP) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunHP.append(float(sun.az*180/3.14))
            altitudeSunHP.append(float(sun.alt*180/3.14))
            gatech.date =str(dateWSW) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunWSW.append(float(sun.az*180/3.14))
            altitudeSunWSW.append(float(sun.alt*180/3.14))

    maxStarArray=[]
    if len(altitudeStar1) > 0:
        maxStarArray.append(max(altitudeStar1))
    else:
        maxStarArray.append(-100)
    if len(altitudeStar2) > 0:
        maxStarArray.append(max(altitudeStar2))
    else:
        maxStarArray.append(-100)
    if len(altitudeStar3) > 0:
        maxStarArray.append(max(altitudeStar3))
    else:
        maxStarArray.append(-100)
    
    maxStar= max(maxStarArray)
   
    if zoom==False:
        ymax = max(altitudeMoon+altitudeSun) +2
        if ymax > maxStar:
            ymax=ymax
        if ymax <= maxStar:
            ymax = maxStar
        xmin = 0
        xmax = 360
        ymin = 0
        ymax = ymax
    if zoom==True:
        xmin = xmin
        xmax = xmax
        ymin = ymin
        ymax = ymax
     
    fig, ax = plt.subplots(1, figsize=(20,9))
    plt.xlim(xmin, xmax)
    plt.ylim(ymin,ymax)
    
    #Horizontprofil
    plt.plot(profilAz,profilAlt , 'k', label="visible horizon")
    plt.plot([0,360],[0,0] , 'k--', label="math. horizon")
  
    
    #Plot Tore
    authorlist= list(anlage["gate"].keys())
    colorlist = ["blue", "green", "orange"]
    for k in range(len(authorlist)):

        for m in range(len(anlage["gate"][authorlist[k]])):
            if k== 0:
                if m == 0:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor=colorlist[k],alpha=0.5 )
                else:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.5)
            if k==1:
                if m == 0:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )
                else:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax],facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )


    #Plot Special Features
    authorlist= list(anlage["specialfeatures"].keys())
    colorlist = ["lightblue",  "lightgreen"]

    
    for k in range(len(authorlist)):
        for m in range(len(anlage["specialfeatures"][authorlist[k]])):
           
            if k==0:
                if m == 0:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor="none", hatch="\\\\", edgecolor="black", linewidth=0.0)
                else:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor="none",hatch="\\\\", edgecolor="black", linewidth=0.0)
            if k==1:
                if m == 0:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor=colorlist[k],alpha=0.7 )
                else:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.7)
     
    if special != "all special events":
        if starname1 != "no star":
            plt.plot(azimuthStar1,altitudeStar1, 'k', label=str(starname1)+', mag: '+str(mag1))
        if starname2 != "no star":
            plt.plot(azimuthStar2,altitudeStar2, 'b', label=str(starname2)+', mag: '+str(mag2))
        if starname3 != "no star":
            plt.plot(azimuthStar3,altitudeStar3, 'r', label=str(starname3)+', mag: '+str(mag3))
        plt.plot(azimuthMoon,altitudeMoon , 'g', label="moon")
        plt.plot(azimuthSun,altitudeSun , 'orange', label="sun")
        
    if special == "all special events":
        plt.plot(azimuthSunFP,altitudeSunFP , 'orange', label="vernal equinox")
        plt.plot(azimuthSunSSW,altitudeSunSSW , 'black', label="summer solstice")
        plt.plot(azimuthSunHP,altitudeSunHP , 'red', label="autumnal equinox")
        plt.plot(azimuthSunWSW,altitudeSunWSW , 'blue', label="winter solstice")
    
    plt.legend(bbox_to_anchor=(0.78, 0.95), loc=2, borderaxespad=0.)
    plt.xlabel('Azimuth [°]')
    plt.ylabel('Altitude [°]')
    plt.title("object: "+anlage["object"]+' (latitude: '+str(anlage["coordinates"]["latitude"])+ '°;  longitude: '+str(anlage["coordinates"]["longitude"])+  '°)\n date: '+str(date)+'\n pressure: '+str(pressureval)+' mBar '+'ecliptic: '+str(eclipt)+'°')
    return plt.show()

In [ ]:
#Test Function II
#AzimuthAltitudeFull("-4700/5/27", "no star", "Rigel", "Sirius", "QuedlinburgI.json", 'all special events', 0,120,-2,3,2110, zoom=False);

In [ ]:
# WIDGET II
import warnings
warnings.filterwarnings('ignore')
out1 = Output()
par1a=Dropdown(
    options=Objects,
    description='object'
)
par2a=Text(
    continuous_update=False,value="-4700",description='year',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par3a=Dropdown(
    continuous_update=False,options=[1,2,3,4,5,6,7,8,9,10,11,12],description='month',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par4a=Dropdown(
    continuous_update=False,options=range(1,32),description='day',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par4b=Dropdown(
    continuous_update=False,options=['no selection', 'all special events', 'vernal_equinox','summer_solstice','autumnal_equinox','winter_solstice'],
    value="no selection",description=' ',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),disabled=False
)
par5a= Dropdown(
    continuous_update=False,options=starsforwidget,value="no star",description='star 1',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par5b= Dropdown(
    continuous_update=False,options=starsforwidget,value="no star",description='star 2',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)

par5c= Dropdown(
    continuous_update=False,options=starsforwidget,value="no star",description='star 3',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par6xmin2=IntSlider(
    value=0,min=0,max=360,step=1,description='az. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6xmax2=IntSlider(
    value=360,min=0,max=360,step=1,description='az. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymin2=FloatSlider(
    value=0,min=-2,max=90,step=0.2,description='alt. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymax2=FloatSlider(
    value=3,min=0,max=90,step=0.2,description='alt. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
parPressure=IntSlider(
    value=1013,min=0,max=2000,step=1,description='p [mBar]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par7a=Button(
   description='RUN'
)
par8a=Button(
   description='RUN with Zoom'
)
def SearchConstellation1(res):
    firstpar =par1a.value
    secondpar = str(par2a.value)
    thirdpar=str(par3a.value)
    fourthpar=str(par4a.value)
    fourthparB=str(par4b.value)
    fivthparA=par5a.value
    fivthparB=par5b.value
    fivthparC=par5c.value
    pressurevar=parPressure.value
    data=secondpar+'/'+thirdpar+'/'+fourthpar
    objectfile=firstpar+".json"
    with out1:
        print("test")
        clear_output()
        AzimuthAltitudeFull(data, str(fivthparA), str(fivthparB), str(fivthparC), objectfile, fourthparB,0, 360, 0, 90, pressurevar, zoom=False)
    return; 
    
def SearchConstellation1Zoom(res):
    firstpar =par1a.value
    secondpar = str(par2a.value)
    thirdpar=str(par3a.value)
    fourthpar=str(par4a.value)
    fourthparB=str(par4b.value)
    fivthparA=par5a.value
    fivthparB=par5b.value
    fivthparC=par5c.value
    sixthxmin2=par6xmin2.value
    sixthxmax2=par6xmax2.value
    sixthymin2=par6ymin2.value
    sixthymax2=par6ymax2.value
    pressurevar=parPressure.value
    data=secondpar+'/'+thirdpar+'/'+fourthpar
    objectfile=firstpar+".json"
    
    with out1:
        clear_output()
        AzimuthAltitudeFull(data,  str(fivthparA), str(fivthparB), str(fivthparC), objectfile, fourthparB, sixthxmin2, sixthxmax2, sixthymin2, sixthymax2,pressurevar, zoom=True)
    return; 
par7a.on_click(SearchConstellation1)
par8a.on_click(SearchConstellation1Zoom)
display(VBox([Label('Select object:'), par1a,HBox([par2a, par3a, par4a]), Label('Select event like solstices or equinox (specific year required)'), par4b,Label('Define pressure: default value is 1013 mBar. The value 0 mBar means airless model without refraction'), parPressure, Label('Select up to three stars'), HBox([ par5a,  par5b, par5c]),  par7a,Label('Define zoom range'), HBox([par6xmin2, par6xmax2,par6ymin2,par6ymax2]),par8a ]))
display(out1)

### Gates
- plot gates distribution of mean azimuth 

In [ ]:
#DEFINE GATES FUNCTION
def Gates(author="Michel"):
    gateslist=list()
    for element in Objects:
        anl=GetJSON(element+".json")
        
        for i in list(anl["gate"].keys()):
            if  i==author:
                gateslist.append(anl["gate"][i])
            d = [item for sublist in gateslist for item in sublist]
    data=list()
    for element in d:
        data.append(np.mean(element))
    bins = np.linspace(math.ceil(min(data)), 
                       math.floor(max(data)),30) # fixed number of bins
    plt.figure(figsize=(15,10))
    plt.xlim([min(data)-5, max(data)+5])

    plt.hist(data, bins=bins, alpha=0.5)
    plt.title('distribution of gates ('+author+'), mean value of gates', fontsize=18)
    plt.xlabel('midpoint of gate [Azimuth [°]]', fontsize=15)
    plt.ylabel('count', fontsize=15)
    plt.show()

In [ ]:
# WIDGET

Authors=["Michel", "Becker", "Staeuble", "Schier", "Schweitzer"]

par1=widgets.Dropdown(
    continuous_update=False,options=Authors, description='',  layout=Layout(margin='10px 10px 10px -50px', min_height='40px', width='700px'),
    disabled=False
)
run1=widgets.Button(description='show')
def ChangeParameter1(res):
    Par1=par1.value
    clear_output()
    display(HBox([Label('author', layout=Layout(width='280px')), par1]),run1)
    display(Gates(author=Par1))
    return;

run1.on_click(ChangeParameter1)
display(HBox([Label('author', layout=Layout(width='280px')), par1]),run1)

## Widget III: star statistics

In [ ]:
# FUNCTION WIDGET III
def FindStarCluster(year, starname, Anlage, maxAltitude):
    gatech = ephem.Observer()
    anlage=GetJSON(Anlage)
    # Korrektur der Werte
    profil=[]
    profilAz=[]
    profilAlt=[]
    az = anlage["profile"]["azimuth"]
    zenitwinkel= anlage["profile"]["altitude"]
    for j in range(len(az)):

        if  float(az[j])>=90 and float(az[j])<360:
            profil.append(((-1*float(az[j]))+360+90,90-float(zenitwinkel[j]) ))

        if float(az[j])<90:
            profil.append(((-1*float(az[j]))+90,90-float(zenitwinkel[j]) ))
    
    profil = sorted(profil,key=lambda l:l[0], reverse=False)
    profilAz=[row[0] for row in profil]
    profilAlt=[row[1] for row in profil]
    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
    
    templist=[]
    azimuthStar=[]
    altitudeStar=[]

    tryv=True
    for j in range(96):

        try:
            gatech.date = str(year)+"/1/1  "+str(j/4.)+":00:00"
            selectedstar=ephem.star(starname)
            selectedstar.compute(gatech)
            templist.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
            templist = sorted(templist,key=lambda l:l[0], reverse=False)
            azimuthStar = [row[0] for row in templist]
            altitudeStar = [row[1] for row in templist]
        except:
            #if j == 1:
            #    print("Star not available on ephem library!")
            tryv=False
            pass
    
    candidates=[]
    d1=[]
    
    authorlist= list(anlage["gate"].keys())
    for k in range(len(authorlist)):
        for m in range(len(anlage["gate"][authorlist[k]])):
            d=np.array(azimuthStar)[np.array(azimuthStar)>anlage["gate"][authorlist[k]][m][0]]
            e=d[d<anlage["gate"][authorlist[k]][m][1]]
            for p in range(len(e)):
                sel=np.where(azimuthStar==e[p])[0][0]
                if len(e)>0 and altitudeStar[sel]>0 and altitudeStar[sel]<maxAltitude :
                    #candidates.append((authorlist[k],e,m+1))
                    candidates.append(m+1)
        
    return candidates;


In [ ]:
# PLOT FUNĆTION

%matplotlib inline
def PlotFunction(var1, var2):
    
    N = len(var2)
    menMeans = var1
    ind = np.arange(N) # the x locations for the groups
    width = 0.59       # the width of the bars
    fig, ax = plt.subplots()
    fig.set_size_inches(26.5, 10.5)
    rects1 = ax.bar(ind, menMeans, width, color='gray')
    # add some text for labels, title and axes ticks
    ax.set_ylabel('#')
    plt.xticks(rotation=90,size = 15)
    ax.set_title('distribution of stars')
    ax.set_xticks(ind )
    ax.set_xticklabels(var2)
    return plt.show()

In [ ]:
# STATISTIC ROUTINE
ObjectsYear=[["QuedlinburgI", -4850],["QuedlinburgII",-4700],["Hopferstadt",-4600],["Bielen",-4750],["Ippesheim",-4800],["Ippesheim_2",-4800],["Kyhna1",-4700], ["Kyhna2",-4700], ["Kyhna3", -4700],["Kyhna4", -4700],["Goseck",-4780], ["Warburg-Daseburg",-4650],["Bochum-Harpen",-4650],["Gneiding",-4750],["Kothingeichendorf",-4750],["Watenstedt",-4700],["Dresden-NickernI",-4800],["Dresden-NickernII",-4800],["Bochow",-4750], ["Eythra2",-4700], ["Dresden3",-4800],["Kuenzing-Unternberg", -4800],["Meisternthal", -4750], ["Mirskofen",-4750], ["Ramsdorf",-4750],["Schmierdorf1",-4750],["Schmierdorf2", -4750],["Stephansposching", -4650], ["Viecht",-4750],["Koetschlitz", -4750]]
def RunStatisticRoutine(maxAltitude, deviation, plot):
    
    possibleCandidate=[]
    possibleCandidates=[]
    listplot=[]
    for l in range(len(ObjectsYear)):
        possibleCandidate=[]
        for m in range(len(stars)):
            f=FindStarCluster(ObjectsYear[l][1], stars[m],ObjectsYear[l][0]+'.json',maxAltitude)
            if len(f)>0:
                if plot==True:
                    AzimuthAltitude(ObjectsYear[l][1], deviation, stars[m], ObjectsYear[l][0]+".json", 2, 360,0,3,zoom=False)
                    possibleCandidate.append(stars[m])
                if plot==False:
                    possibleCandidate.append(stars[m])
        possibleCandidates.append((ObjectsYear[l][0],possibleCandidate))
    result=dict(possibleCandidates)  
    ######################
    if plot==False:
        anl=list(result.keys())
        for j in range(len(anl)):
            print("--------------------------------------------------------------------------------")
            print(str(anl[j])+": "+str(result[anl[j]]))
    ######################
    d2=[]
    d3=[]
    d4=[]
    anlagen=list(result.keys())
    for j in range(len(anlagen)):
        d2.append(result[anlagen[j]])
    d3=[item for sublist in d2 for item in sublist]
    d4 = sorted(set(d3))
    res = Counter(d3)
    
    return PlotFunction(res.values(),d4)

In [ ]:
#WIDGET III
import warnings
warnings.filterwarnings('ignore')

out2 = Output()


par23=Text(
    continuous_update=False,
    value="2",
    description='Alt[°]',
    layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par33=Dropdown(
    continuous_update=False,
    options=[10,20,30,40,50,60,70,80,90,100,110,120, 130, 150, 200],
    value=100,
    description='dev[y]',
    layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False

)

parrun1=Button(
   
    description='RUN WITHOUT PLOTS'
)

parrun2=Button(
   
    description='RUN WITH PLOTS'
)

def Statistics1(res):
  
    secondpar = par23.value
    thirdpar=par33.value
  
    
    with out2:
        clear_output()
        RunStatisticRoutine(int(secondpar), int(thirdpar), False)
    return; 

def Statistics2(res):
  
    secondpar = par23.value
    thirdpar=par33.value
  
    
    with out2:
        clear_output()
        RunStatisticRoutine(int(secondpar), int(thirdpar), True)
    return; 
parrun1.on_click(Statistics1)
parrun2.on_click(Statistics2)

display(VBox([Label('maximal Altitude'), par23, Label('deviation from year'), par33, parrun1 , parrun2]))
display(out2)

## Widget IV: constellations for 4700 BC and 2000

In [ ]:
# ESTIMATE CONSTELLATION
constell=[]
for k in range(len(stars)):
    try:
         constell.append((stars[k],constellation(ephem.star(stars[k],'2000/1/1'))[1]))
    except:
        pass
constell;
s=constell
def const(const,year=None,epoch='2000',title=None):
    s = []
    for star in const:
        s.append(ephem.star(star))
    for i in range(len(s)):
        if(year!=None):
            s[i].compute(year,epoch=epoch)
        else:
            s[i].compute(epoch=epoch)
    tau = 2.0 * pi
    degree = tau / 360.0
    hour = tau / 24.0
    ra_list = [star.a_ra / hour for star in s]
    dec_list = [star.a_dec / degree for star in s]

    mag_array = np.array([star.mag for star in s])
    size_array = (5 - mag_array) ** 1.5 * 16
    plt.scatter(ra_list,dec_list,size_array, label=year, marker='o')
    if(title!=None):
        plt.title(title)
    plt.legend(bbox_to_anchor=(0.90, 0.95), loc=2, borderaxespad=0.)
    return s, ra_list, dec_list
constellSt=[]
for j in range(len(constell)):
    constellSt.append(constell[j][1])
constellSt = list(set(constellSt))
constellSt;

In [ ]:
# FUNCTION IV
def CompareConstellation(currentConstell):
    starconstell=[]
    plt.figure(figsize=(20,8));
    ax = plt.axes([0.025, 0.025, 0.95, 0.95])
    ax.grid(which='major', axis='x', linewidth=0.75, linestyle='-', color='0.75')
    ax.grid(which='minor', axis='x', linewidth=0.25, linestyle='-', color='0.75')
    ax.grid(which='major', axis='y', linewidth=0.75, linestyle='-', color='0.75')
    ax.grid(which='minor', axis='y', linewidth=0.25, linestyle='-', color='0.75')
    plt.xlabel('right ascension [h]')
    plt.ylabel('declination[°]')
    for j in range(len(constell)):
        if constell[j][1]==currentConstell:
            starconstell.append((constell[j][0]))
    f1=const(starconstell,year='-4700',title=currentConstell)
    const(starconstell,year='2000',title=currentConstell)
    for j in range(len(starconstell)):
        ax.annotate(starconstell[j], xy=(f1[1][j], f1[2][j]), xytext=(f1[1][j], f1[2][j]+0.001*f1[2][j]))

In [ ]:
#WIDGET IV
import warnings
warnings.filterwarnings('ignore')
out3 = Output()
parA=Dropdown(
    continuous_update=False,options=constellSt,value="Andromeda",description='const',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
parrun4 = Button(
  description='RUN'
)
def Procedure(res):
    firstpar = parA.value
    clear_output()
    display(VBox([Label('Constellation'), parA, parrun4]))
    CompareConstellation(firstpar)
    return; 
parrun4.on_click(Procedure)
display(VBox([Label('Constellation'), parA, parrun4]))
display(out3)

## Widget V: moon turn

In [2]:
# FUNCTION V
def AzimuthAltitudeMoon(date1, date2, Anlage, special, xmin, xmax, ymin, ymax, pressureval, zoom):
    gatech = ephem.Observer()
    gatech.pressure=pressureval
    anlage=GetJSON(Anlage)
    # Korrektur der Werte
    profil=[]
    profilAz=[]
    profilAlt=[]
    az = anlage["profile"]["azimuth"]
    zenitwinkel= anlage["profile"]["altitude"]

    for j in range(len(az)):
   
        if  float(az[j])>=90 and float(az[j])<360:
            profil.append(((-1*float(az[j]))+360+90,90-float(zenitwinkel[j]) ))
            
        if float(az[j])<90:
            profil.append(((-1*float(az[j]))+90,90-float(zenitwinkel[j]) ))
            

    profil = sorted(profil,key=lambda l:l[0], reverse=False)
    profilAz=[row[0] for row in profil]
    profilAlt=[row[1] for row in profil]
    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
   
    azimuthMoon=[]
    altitudeMoon=[]
    azimuthMoon1=[]
    altitudeMoon1=[]
  
    for j in range(96):
        gatech.date =str(date1) +' '+str(j/4.)+":00:00"
        
        #Moon
        moon=ephem.Moon()
        moon.compute(gatech)
        azimuthMoon.append(float(moon.az*180/3.14))
        altitudeMoon.append(float(moon.alt*180/3.14))
        #Moon
        gatech.date =str(date2) +' '+str(j/4.)+":00:00"
        moon1=ephem.Moon()
        moon1.compute(gatech)
        azimuthMoon1.append(float(moon1.az*180/3.14))
        altitudeMoon1.append(float(moon1.alt*180/3.14))
    
    fig, ax = plt.subplots(1, figsize=(20,9))
    plt.xlim(0, 360)
    plt.ylim(0,80)
    
    #Horizontprofil
    plt.plot(profilAz,profilAlt , 'k', label="visible horizon")
    plt.plot([0,360],[0,0] , 'k--', label="math. horizon")
  
    ymax=80
    #Plot Tore
    authorlist= list(anlage["gate"].keys())
    colorlist = ["blue", "green", "orange"]
    for k in range(len(authorlist)):

        for m in range(len(anlage["gate"][authorlist[k]])):
            if k== 0:
                if m == 0:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor=colorlist[k],alpha=0.5 )
                else:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.5)
            if k==1:
                if m == 0:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )
                else:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax],facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )


    #Plot Special Features
    authorlist= list(anlage["specialfeatures"].keys())
    colorlist = ["lightblue",  "lightgreen"]

    
    for k in range(len(authorlist)):
        for m in range(len(anlage["specialfeatures"][authorlist[k]])):
           
            if k==0:
                if m == 0:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor="none", hatch="\\\\", edgecolor="black", linewidth=0.0)
                else:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor="none",hatch="\\\\", edgecolor="black", linewidth=0.0)
            if k==1:
                if m == 0:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor=colorlist[k],alpha=0.7 )
                else:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.7)
     
    
    plt.plot(azimuthMoon,altitudeMoon , 'g', label="moon")
    plt.plot(azimuthMoon1,altitudeMoon1 , 'g')
    plt.legend(bbox_to_anchor=(0.78, 0.95), loc=2, borderaxespad=0.)
    plt.xlabel('Azimuth [°]')
    plt.ylabel('Altitude [°]')
    plt.title("object: "+anlage["object"]+' (latitude: '+str(anlage["coordinates"]["latitude"])+ '°;  longitude: '+str(anlage["coordinates"]["longitude"])+  '°)\n date: '+str(date1)+'\n date: '+str(date2))
    return plt.show()

In [3]:
# CHECK 
def CheckMoonTurn(Anlage,year):
    anlage=GetJSON(Anlage+".json")
    gatech = ephem.Observer()
    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
    
    moon = ephem.Moon()
    centurylist=[]
    yearlist=[]
    maxextrema=[]
    cvcur=-10000
    cvcur1=10000
    for p in range(year-10, year+10):
        for m in range(1,12):
            moonlist=[]
            datelist=[]
            for j in range(30):
                gatech.date = str(p)+'/'+str(m)+'/'+str(j)+' 00:00'
                rise=gatech.previous_rising(ephem.Moon())
                gatech.date = str(rise)
                moon.compute(gatech)
                moonlist.append(float(moon.az*180/3.14))
                datelist.append((p,m,j))
            cv=max(moonlist)-min(moonlist)
            if cv>cvcur:
                cvcur=cv
                year=str(p)
                month=str(m)
                daymax=str(moonlist.index(max(moonlist)))
                daymin=str(moonlist.index(min(moonlist)))
            if cv<cvcur1:
                cvcur1=cv
                yearMin=str(p)
                monthMin=str(m)
                daymaxMin=str(moonlist.index(max(moonlist)))
                dayminMin=str(moonlist.index(min(moonlist)))
    print("maximal moon turn:")
    AzimuthAltitudeMoon(year+"/"+month+"/"+daymin, year+"/"+month+"/"+daymax, Anlage+".json", 'no selection', 0,120,-2,3,2110, zoom=False);
    print("minimal moon turn:")
    AzimuthAltitudeMoon(yearMin+"/"+monthMin+"/"+dayminMin, yearMin+"/"+monthMin+"/"+daymaxMin, Anlage+".json", 'no selection', 0,120,-2,3,2110, zoom=False);
    #return cvcur, year, month, daymax, daymin;

In [4]:
#WIDGET V
import warnings
warnings.filterwarnings('ignore')
out4 = Output()
parB=Dropdown(
    continuous_update=False,options=Objects,value="QuedlinburgI",description='obj',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
parC=Text(
    continuous_update=False,value="-4700",description='year',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)

parrun5 = Button(
  description='RUN'
)

def Procedure(res):
    firstpar = parB.value
    secondpar = int(parC.value)
    clear_output()
    display(VBox([Label('Object'), parB, Label("year"), parC, parrun5]))
    CheckMoonTurn(firstpar, secondpar)
    return; 
parrun5.on_click(Procedure)
display(VBox([Label('Object'), parB, Label("year"), parC, parrun5]))
display(out4)

NameError: name 'Output' is not defined